*Note*: If you want to run this script on your machine. Please run the script `install.ipynb` first

## Load data

In [84]:
import pandas as pd

# Load data from excel
# load all sheets with sheet_name=None
# mark DATE as date type
sheets = pd.read_excel("MoMo-Case-challenge-for-DA.xlsx", sheet_name=None)

# transDf contain the data of `Transactions` table
transDf = sheets["Data. Transactions"]
transDf['DATE'] = pd.to_datetime(transDf['DATE'])

# transDf contain the data of `Loyalty Points` table
pointDf = sheets["Data. Loyalty Points"]

# benefitDf contain the data of `Loyalty Benefits` table
benefitDf = sheets["Data. Loyalty benefits"]

pointDf.head()

,Service Group,Point Mechanism,Maximum Point Per Trans
0,supermarket,1 points/1000VND GMV,500 points
1,data,10 points/1000VND GMV,1000 points
2,cvs,1 points/1000VND GMV,300 points
3,marketplace,1 points/1000VND GMV,500 points
4,Coffee chains and Milk tea,1 points/1000VND GMV,500 points


Create Ranking table:

In [85]:
rankDf = pd.DataFrame({'Class_ID': [1, 2, 3, 4],
        'Rank_name': ['STANDARD', 'SILVER', 'GOLD', 'DIAMOND'],
        'Loyalty_Points': [999, 1999, 4999, 5000]})
rankDf.head()

,Class_ID,Rank_name,Loyalty_Points
0,1,STANDARD,999
1,2,SILVER,1999
2,3,GOLD,4999
3,4,DIAMOND,5000


## Normalize data

Convert `Point Mechanism` to number:

In [86]:
pointDf['Point Mechanism'] = pd.to_numeric(pointDf['Point Mechanism'].str.replace(' points/1000VND GMV', ''), errors='coerce')
pointDf.head()

,Service Group,Point Mechanism,Maximum Point Per Trans
0,supermarket,1,500 points
1,data,10,1000 points
2,cvs,1,300 points
3,marketplace,1,500 points
4,Coffee chains and Milk tea,1,500 points


Convert `Maximum Point Per Trans` to number:

In [87]:
pointDf['Maximum Point Per Trans'] = pd.to_numeric(pointDf['Maximum Point Per Trans'].str.replace(' points', ''), errors='coerce')
pointDf.head()

,Service Group,Point Mechanism,Maximum Point Per Trans
0,supermarket,1,500
1,data,10,1000
2,cvs,1,300
3,marketplace,1,500
4,Coffee chains and Milk tea,1,500


## Part 1

### Combined with the `Loyalty Points` table, add a column `Loyalty_Points` in the `Transactions` table with the given rules. 

In [88]:
mergedDf = pd.merge(transDf, pointDf, on='Service Group')
mergedDf['Loyalty_Points'] = mergedDf.apply(lambda x: min(x['GMV'] * x['Point Mechanism'], x['Maximum Point Per Trans']), axis=1)

transDf['Loyalty_Points'] = mergedDf['Loyalty_Points']
transDf.head()

,DATE,Order_id,NEWVERTICAL_Merchant,MerchantID,User_id,GMV,Service Group,Loyalty_Points
0,2021-01-01,8733622706,Marketplace,37,61386143,100000,marketplace,500
1,2021-01-01,8726857991,Supermarket,9,48453125,5000,supermarket,500
2,2021-01-01,8737326894,Supermarket,9,49921027,106600,supermarket,500
3,2021-01-01,8732579078,supermarket,9,46022523,270000,supermarket,500
4,2021-01-01,8725567343,CVS,8,44014594,68000,cvs,500


### Create another table named `Loyalty Ranking` which must include columns named `Rank_name` and `Calculated_points` to calculate the Rank of each user on a daily basis.

In [89]:
def get_rank(point):
    ranks = rankDf[rankDf['Loyalty_Points'] < 3000].sort_values(by=['Loyalty_Points'], ascending=False)
    highest_rank = ranks.iloc[0]['Rank_name'] if len(ranks) >= 1 else rankDf[0]['Rank_name']
    return highest_rank

# add Calculated_points column
userRankDf = transDf.groupby(['DATE', 'User_id'])['Loyalty_Points'].sum().reset_index()
userRankDf.rename(columns={'Loyalty_Points': 'Calculated_points'}, inplace=True)

# add Rank_name column
userRankDf['Rank_name'] = userRankDf['Calculated_points'].apply(get_rank)

userRankDf.head()

,DATE,User_id,Calculated_points,Rank_name
0,2021-01-01,11368352,500,SILVER
1,2021-01-01,31058664,500,SILVER
2,2021-01-01,36415344,500,SILVER
3,2021-01-01,44014594,1000,SILVER
4,2021-01-01,46022523,500,SILVER


### At the end of Mar 2022, how many users achieved the GOLD rank?


In [98]:
end_of_mar_2022 = pd.Timestamp('2022-03-31')
rank = 'GOLD'
result = userRankDf[(userRankDf['DATE'] <= end_of_mar_2022) & (userRankDf['Rank_name'] == rank)]['User_id'].nunique()
print(f'At the end of Mar 2022, "{result}" users achieved the {rank} rank.')


At the end of Mar 2022, "0" users achieved the GOLD rank.


## Save data to files

In [91]:
transDf.to_excel('Transactions.xlsx', index=False)
userRankDf.to_excel('Loyalty Ranking.xlsx', index=False)